# Apriori Algorithm From Scratch with Transaction Reduction Improvement
## Author: Sayan Swar;     Email: sswar@ur.rochester.edu
### Data Set: https://archive.ics.uci.edu/ml/datasets/adult

 ____________________________________________________________

In [130]:
import pandas as pd
import numpy as np
import math as mt
import itertools 
from itertools import product
import time
import collections

In [144]:
colnames = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship',
            'race','sex','capital-gain','capital-loss','hours-per-week','native-country','Income']

#reading the data file
df = pd.read_csv('adult.data', header=None, names=colnames)

#removing all unnecessary columns
df = df.drop('age', axis=1)
df = df.drop('fnlwgt', axis=1)
df = df.drop('capital-gain', axis=1)
df = df.drop('capital-loss', axis=1)
df = df.drop('hours-per-week', axis=1)
df = df.drop('education-num', axis=1)

#getting the count of each individual item in the data set
itemset1 = {}
for cols in list(df):
    itemset1.update(df[cols].value_counts())

#creating a new column called attribute set in the dataframe which holds all the concatenated values of a single
#transaction in a list
df['attribute_set'] = df.values.tolist()
#df['isFlaggedToDelete'] = 1
#df.head(5)

In [149]:
#deciding the minimum support to be 0.5. Calulating the value of support count based on min support.
min_support = 0.5
number_of_transaction = len(df)
support_count = number_of_transaction * min_support
support_count,number_of_transaction

(16280.5, 32561)

In [150]:
#calculating itemset 1. Removing all those items which doesnt follow the min support count.
#the calculated itemset 1 will be used to calculate next itemsets.
candidateset_1 =  pd.DataFrame(list(itemset1.items()), columns = ['Items','Counts'])

candidateset_1_with_frequentitems = candidateset_1[candidateset_1['Counts']>=support_count]
candidateset_1_with_frequentitems = candidateset_1_with_frequentitems.sort_values(by=['Counts'],ascending=0)
candidateset_1_with_frequentitems['Items'] = candidateset_1_with_frequentitems['Items'].astype('str')
candidateset_1_with_frequentitems_set = set(candidateset_1_with_frequentitems.loc[:,'Items'])
candidateset_1_with_frequentitems_set_dict = dict(candidateset_1_with_frequentitems.values)
candidateset_1_with_frequentitems_set_dict

{' United-States': 29170,
 ' White': 27816,
 ' <=50K': 24720,
 ' Private': 22696,
 ' Male': 21790}

In [151]:
#defining a function which can create all the required combinatons (Lk ad Ck) from itemset 2.
def find_k_itemsets(itemset,itemsetcount,df,support_count):
    
    candidateset_with_frequentitems_list = sorted(list(itemset.keys()))
    itemset_count = itemsetcount
    i=0
    itemsetk_set = set()
    len_of_frequentsetk = len(candidateset_with_frequentitems_list)
    for i in range(len_of_frequentsetk):
        for j in range(i+1,len_of_frequentsetk):
            if j<=len_of_frequentsetk:
                if itemset_count<3:
                    l1 = [candidateset_with_frequentitems_list[i]]
                    l2 = [candidateset_with_frequentitems_list[j]]
                else:
                    l1 = sorted(candidateset_with_frequentitems_list[i])
                    l2 = sorted(candidateset_with_frequentitems_list[j])
                joinitems = set(l2).union(set(l1))
                joinitems = sorted(joinitems)
                if len(joinitems) == itemset_count:
                    itemsetk_set.add(tuple(joinitems))
        
##creating subsets and checking if each of the subsets were frequent in the previous step or not
##if not frquent in the previous step, then we are completely discarding the new itemset...........................
    itemsetk_set_improved = set()
    if itemset_count>2:
        for item in itemsetk_set:
            x = list(itertools.combinations(item, itemset_count-1))
            check = all(things in sorted(list(candidateset_with_frequentitems_list)) for things in sorted(x))
            if check==True:
                itemsetk_set_improved.add(tuple(item))
    else:
        itemsetk_set_improved = itemsetk_set
##subset creation ends..................................................................................................
    
    len_itemsetk = len(itemsetk_set_improved)
    frequecy_count = 0
    frequency_itemsetk = {}
    ind = list() #transaction reduction code part
    for i in range(0,len_itemsetk):
        frequecy_count = 0
        for j in df.index:
            check = all(item in df.loc[j,'attribute_set'] for item in list(itemsetk_set_improved)[i])
            if check is True:
                frequecy_count += 1
                frequency_itemsetk[list(itemsetk_set_improved)[i]] = frequecy_count
                ##df.loc[j,'isFlaggedToDelete'] = 0
                ind.append(j) #transaction reduction code part as an improvement
    
    frequency_itemsetk_filtered = {key:value for (key, value) in frequency_itemsetk.items() if value >= support_count}
    return frequency_itemsetk_filtered,list(set(ind)) #transaction reduction code part, returning ind




In [152]:
start = time.time()
frequency_itemsetk_filtered_final_list = {}
frequency_itemsetk_filtered_final_list.update(candidateset_1_with_frequentitems_set_dict)
temp_dict = {}
temp_dict = candidateset_1_with_frequentitems_set_dict
updated_df = df
for i in range (2,number_of_transaction+1):
    if len(temp_dict)>1:
        print(len(updated_df))
        k_filtered_itemset,ind = find_k_itemsets(temp_dict,i,updated_df,support_count) #transaction reduction code part, returning ind
        updated_df = updated_df.loc[ind] #transaction reduction code part
        temp_dict = {}
        temp_dict = k_filtered_itemset
        frequency_itemsetk_filtered_final_list.update(k_filtered_itemset)
        i+=1
    else:
        break

end = time.time()        
end - start

32561
32290
28820


5.628982067108154

In [153]:
frequency_itemsetk_filtered_final_list

{' United-States': 29170,
 ' White': 27816,
 ' <=50K': 24720,
 ' Private': 22696,
 ' Male': 21790,
 (' Private', ' White'): 19404,
 (' United-States', ' White'): 25621,
 (' Private', ' United-States'): 20135,
 (' Male', ' White'): 19174,
 (' <=50K', ' Private'): 17733,
 (' <=50K', ' White'): 20699,
 (' <=50K', ' United-States'): 21999,
 (' Male', ' United-States'): 19488,
 (' Private', ' United-States', ' White'): 17728,
 (' <=50K', ' United-States', ' White'): 18917,
 (' Male', ' United-States', ' White'): 17653}

In [90]:
len(frequency_itemsetk_filtered_final_list)

16